In [1]:
import cobra

In [2]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated.xml")

In [5]:
solution = []
# glucose
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of glucose can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
10.0 EX_glc__D_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10 mmol of glucose can make 0.440 g of biomass


In [6]:
solution.fluxes

12DGR140tipp          0.0
13PPDH                0.0
1P2CBXLCYCL           0.0
1P2CBXLR              0.0
23CTI1                0.0
                     ... 
EX_guaiacol_e         0.0
guaiacol_transport    0.0
GUADEM                0.0
Growth_Phenol         0.0
Growth_Glucose        0.0
Name: fluxes, Length: 3019, dtype: float64

In [7]:
for r in model.metabolites.get_by_id('glc__D_e').reactions:
    print(solution.fluxes[r.id], r)

-10.0 EX_glc__D_e: glc__D_e <=> 
0.0 GLCtex: glc__D_e <=> glc__D_p
0.0 GLCabc: atp_c + glc__D_e + h2o_c --> adp_c + glc__D_c + h_c + pi_c
0.0 BG_CELLB: cellb_e + h2o_e --> 2.0 glc__D_e
10.0 GLCpts: glc__D_e + pep_c --> g6p_c + pyr_c


In [9]:
for r in model.metabolites.get_by_id('g6p_c').reactions:
    if model.metabolites.get_by_id('f6p_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

9.909707708546922 PGI: g6p_c <=> f6p_c


In [14]:
for r in model.metabolites.get_by_id('f6p_c').reactions:
    if model.metabolites.get_by_id('fdp_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

9.81293243056751 PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
0.0 FBP: fdp_c + h2o_c --> f6p_c + pi_c


In [15]:
for r in model.metabolites.get_by_id('fdp_c').reactions:
    if model.metabolites.get_by_id('dhap_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

9.81293243056751 FBA: fdp_c <=> dhap_c + g3p_c
0.0 FRUK: atp_c + f1p_c --> adp_c + fdp_c + h_c
0.0 FBP: fdp_c + h2o_c --> f6p_c + pi_c
9.81293243056751 PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c


In [16]:
for r in model.metabolites.get_by_id('dhap_c').reactions:
    if model.metabolites.get_by_id('g3p_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 TGBPA: tagdp__D_c <=> dhap_c + g3p_c
9.81293243056751 FBA: fdp_c <=> dhap_c + g3p_c
9.532243206603438 TPI: dhap_c <=> g3p_c


In [19]:
for r in model.metabolites.get_by_id('g3p_c').reactions:
    if model.metabolites.get_by_id('13dpg_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 GAPDi_nadp: 13dpg_c + h_c + nadph_c --> g3p_c + nadp_c + pi_c
18.82315537886211 GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c


In [22]:
for r in model.metabolites.get_by_id('13dpg_c').reactions:
    if model.metabolites.get_by_id('3pg_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

-18.82315537886211 PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c


In [24]:
for r in model.metabolites.get_by_id('3pg_c').reactions:
    if model.metabolites.get_by_id('2pg_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

-18.14741007187831 PGM: 2pg_c <=> 3pg_c


In [25]:
for r in model.metabolites.get_by_id('2pg_c').reactions:
    if model.metabolites.get_by_id('pep_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

18.14741007187831 ENO: 2pg_c <=> h2o_c + pep_c


In [34]:
rxns = []
for r in model.metabolites.get_by_id('pep_c').reactions:
    if model.metabolites.get_by_id('pep_c') in r.reactants and model.metabolites.get_by_id('pyr_c') in r.products:
        print(r.id)
        rxns.append(r.id)
    elif model.metabolites.get_by_id('pep_c') in r.products and model.metabolites.get_by_id('pyr_c') in r.reactants:
        print(f'reverse_{r.id}')
        rxns.append(f'reverse_{r.id}')

print(' or '.join(rxns))

ACMANApts
reverse_PPS
PYK
FRUptspp
MALTptspp
DHAPT
SUCpts
GLCptspp
MNLpts
FRUpts2pp
SBTpts
SUCptspp
GLCpts
ACGApts
FRUpts2
GALpts
MALTpts
GLCADpts
MANptspp
FRUpts
ARBTpts
TREptspp
GALTpts
TREpts
ASCBpts
GAMpts
MANpts
ACGAptspp
FUCpts
ACMANApts or reverse_PPS or PYK or FRUptspp or MALTptspp or DHAPT or SUCpts or GLCptspp or MNLpts or FRUpts2pp or SBTpts or SUCptspp or GLCpts or ACGApts or FRUpts2 or GALpts or MALTpts or GLCADpts or MANptspp or FRUpts or ARBTpts or TREptspp or GALTpts or TREpts or ASCBpts or GAMpts or MANpts or ACGAptspp or FUCpts


In [47]:
for r in model.metabolites.get_by_id('6pgc_c').reactions:
    if model.metabolites.get_by_id('ru5p__D_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 GND: 6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c


In [49]:
for r in model.metabolites.get_by_id('ru5p__D_c').reactions:
    if model.metabolites.get_by_id('xu5p__D_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

-0.03246954845672967 RPE: ru5p__D_c <=> xu5p__D_c


In [54]:
for r in model.metabolites.get_by_id('xu5p__D_c').reactions:
    if model.metabolites.get_by_id('g3p_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 PKETX: pi_c + xu5p__D_c --> actp_c + g3p_c + h2o_c
0.5140384197443857 TKT1: r5p_c + xu5p__D_c <=> g3p_c + s7p_c
-0.5465079682011154 TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c


In [56]:
for r in model.metabolites.get_by_id('e4p_c').reactions:
    if model.metabolites.get_by_id('f6p_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.5140384197443857 TALA: g3p_c + s7p_c <=> e4p_c + f6p_c
0.0 PKETF: f6p_c + pi_c --> actp_c + e4p_c + h2o_c
-0.5465079682011154 TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c


In [60]:
for r in model.metabolites.get_by_id('g6p_c').reactions:
    if model.metabolites.get_by_id('6pgl_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 G6PDH2r: g6p_c + nadp_c <=> 6pgl_c + h_c + nadph_c


In [58]:
for r in model.metabolites.get_by_id('6pgl_c').reactions:
    if model.metabolites.get_by_id('6pgc_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 PGL: 6pgl_c + h2o_c --> 6pgc_c + h_c


In [62]:
for r in model.metabolites.get_by_id('6pgc_c').reactions:
    if model.metabolites.get_by_id('2ddg6p_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 EDD: 6pgc_c --> 2ddg6p_c + h2o_c


In [64]:
for r in model.metabolites.get_by_id('2ddg6p_c').reactions:
    if model.metabolites.get_by_id('g3p_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 EDA: 2ddg6p_c --> g3p_c + pyr_c


In [65]:
for r in model.metabolites.get_by_id('pyr_c').reactions:
    if model.metabolites.get_by_id('accoa_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

14.306146518908998 PDH: coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c


In [68]:
for r in model.metabolites.get_by_id('oaa_c').reactions:
    if model.metabolites.get_by_id('cit_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 CITL: cit_c --> ac_c + oaa_c
13.888769300541648 CS: accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c


In [69]:
for r in model.metabolites.get_by_id('cit_c').reactions:
    if model.metabolites.get_by_id('icit_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

13.888769300541648 ACONT: cit_c <=> icit_c


In [73]:
for r in model.metabolites.get_by_id('icit_c').reactions:
    print(solution.fluxes[r.id], r)

0.0 ICITt4pp: icit_p + na1_p --> icit_c + na1_c
13.888769300541648 ICDHx: icit_c + nad_c <=> akg_c + co2_c + nadh_c
0.0 ICDHyr: icit_c + nadp_c <=> akg_c + co2_c + nadph_c
0.0 ICITtupp: icit_p --> icit_c
0.0 ICITt_kt: h_p + icit_p --> h_c + icit_c
13.888769300541648 ACONT: cit_c <=> icit_c
0.0 ICL: icit_c --> glx_c + succ_c
0.0 ACONTb: acon_C_c + h2o_c <=> icit_c


In [76]:
for r in model.metabolites.get_by_id('akg_c').reactions:
    if model.metabolites.get_by_id('succoa_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

-1.2957208093638117 KGD2: co2_c + 2.0 fdxrd_c + h_c + succoa_c <-- akg_c + coa_c + 2.0 fdxox_c
12.128842166861293 AKGDH: akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c


In [82]:
rxns = []
for r in model.metabolites.get_by_id('succoa_c').reactions:
    if model.metabolites.get_by_id('succoa_c') in r.reactants and model.metabolites.get_by_id('succ_c') in r.products:
        print(r.id)
        rxns.append(r.id)
    elif model.metabolites.get_by_id('succoa_c') in r.products and model.metabolites.get_by_id('succ_c') in r.reactants:
        print(f'reverse_{r.id}')
        rxns.append(f'reverse_{r.id}')
        
print(' or '.join(rxns))

SUCBZT2
OCOAT1
SUCBZT1
reverse_SUCOAS
BZSCT
OCOAT4
3OADPCOAT
SUCBZT2 or OCOAT1 or SUCBZT1 or reverse_SUCOAS or BZSCT or OCOAT4 or 3OADPCOAT


In [85]:
rxns = []
for r in model.metabolites.get_by_id('succ_c').reactions:
    if model.metabolites.get_by_id('succ_c') in r.reactants and model.metabolites.get_by_id('fum_c') in r.products:
        print(r.id)
        rxns.append(r.id)
    elif model.metabolites.get_by_id('succ_c') in r.products and model.metabolites.get_by_id('fum_c') in r.reactants:
        print(f'reverse_{r.id}')
        rxns.append(f'reverse_{r.id}')
        
print(' or '.join(rxns))

reverse_FRD3
reverse_FRD2
reverse_ASPO5
reverse_FRD
SUCD1
reverse_FRDx
reverse_FRD3 or reverse_FRD2 or reverse_ASPO5 or reverse_FRD or SUCD1 or reverse_FRDx


In [88]:
for r in model.metabolites.get_by_id('fum_c').reactions:
    if model.metabolites.get_by_id('mal__L_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

13.883765345794314 FUM: fum_c + h2o_c <=> mal__L_c


In [90]:
for r in model.metabolites.get_by_id('mal__L_c').reactions:
    if model.metabolites.get_by_id('oaa_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

13.2446921428716 DMALRED: fad_c + mal__L_c --> fadh2_c + oaa_c
0.0 MDH2: mal__L_c + q8_c --> oaa_c + q8h2_c
0.6873936732565692 MDH3: mal__L_c + mqn8_c --> mql8_c + oaa_c
0.0 MDH3_1: mal__L_c + mqn7_c --> mql7_c + oaa_c
0.0 MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c


In [91]:
for r in model.metabolites.get_by_id('icit_c').reactions:
    if model.metabolites.get_by_id('succ_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 ICL: icit_c --> glx_c + succ_c


In [94]:
for r in model.metabolites.get_by_id('glx_c').reactions:
    if model.metabolites.get_by_id('accoa_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.04832047033385484 MALS: accoa_c + glx_c + h2o_c --> coa_c + h_c + mal__L_c


In [97]:
for r in model.metabolites.get_by_id('pyr_c').reactions:
    if model.metabolites.get_by_id('mal__L_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 ME1: mal__L_c + nad_c --> co2_c + nadh_c + pyr_c
0.0 ME2: mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c


In [98]:
for r in model.metabolites.get_by_id('pep_c').reactions:
    if model.metabolites.get_by_id('oaa_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

1.2471131654981158 PPC: co2_c + h2o_c + pep_c --> h_c + oaa_c + pi_c
0.0 PEPCK_re: gtp_c + oaa_c --> co2_c + gdp_c + pep_c


In [100]:
for r in model.metabolites.get_by_id('nadh_c').reactions:
    if model.metabolites.get_by_id('nadph_c') in r.metabolites:
        print(solution.fluxes[r.id], r)

0.0 NADTRHD: nad_c + nadph_c --> nadh_c + nadp_c


In [112]:
for r in model.metabolites.get_by_id('co2_e').reactions:
#     if model.metabolites.get_by_id('nadph_c') in r.metabolites:
    print(solution.fluxes[r.id], r)

-42.89713409954565 CO2t: co2_e <=> co2_c
42.89713409954565 EX_co2_e: co2_e --> 
0.0 CO2tex: co2_e <=> co2_p


In [105]:
model.reactions.get_by_id('O2t')

KeyError: 'O2t'

In [ ]:
model.metabolites.